In [97]:
import pandas as pd
import numpy as np
import janitor

In [98]:
# Reading the data into dataframes
vote = pd.read_csv("countypres_2000-2020.csv")
cost = pd.read_csv("./cost_of_living_us.csv")
vote = vote[vote.year == 2020]
vote

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
50524,2020,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,7503,27770,20220315,TOTAL
50525,2020,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,OTHER,OTHER,429,27770,20220315,TOTAL
50526,2020,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,19838,27770,20220315,TOTAL
50527,2020,ALABAMA,AL,BALDWIN,1003.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,24578,109679,20220315,TOTAL
50528,2020,ALABAMA,AL,BALDWIN,1003.0,US PRESIDENT,OTHER,OTHER,1557,109679,20220315,TOTAL
...,...,...,...,...,...,...,...,...,...,...,...,...
72612,2020,WYOMING,WY,WASHAKIE,56043.0,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,3245,4032,20220315,TOTAL
72613,2020,WYOMING,WY,WESTON,56045.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,360,3560,20220315,TOTAL
72614,2020,WYOMING,WY,WESTON,56045.0,US PRESIDENT,JO JORGENSEN,LIBERTARIAN,46,3560,20220315,TOTAL
72615,2020,WYOMING,WY,WESTON,56045.0,US PRESIDENT,OTHER,OTHER,47,3560,20220315,TOTAL


In [92]:
vote = vote.groupby(["year","state", "county_name", "county_fips", "party", "totalvotes"])["candidatevotes"].sum().reset_index()

,state,county_name,party,candidatevotes,totalvotes,county_fips
50524,ALABAMA,AUTAUGA,DEMOCRAT,7503,27770,1001.0
50525,ALABAMA,AUTAUGA,OTHER,429,27770,1001.0
50526,ALABAMA,AUTAUGA,REPUBLICAN,19838,27770,1001.0
50527,ALABAMA,BALDWIN,DEMOCRAT,24578,109679,1003.0
50528,ALABAMA,BALDWIN,OTHER,1557,109679,1003.0
...,...,...,...,...,...,...
72612,WYOMING,WASHAKIE,REPUBLICAN,3245,4032,56043.0
72613,WYOMING,WESTON,DEMOCRAT,360,3560,56045.0
72614,WYOMING,WESTON,LIBERTARIAN,46,3560,56045.0
72615,WYOMING,WESTON,OTHER,47,3560,56045.0


In [93]:
vote.party.unique()
# There's not so many parties that we can't reduce the row count by combining the parties onto one county

vcounty = pd.pivot_table(vote, index=["state", "county_name", "totalvotes", "county_fips"], columns="party", values=["candidatevotes"])
vcounty = vcounty.droplevel(0, axis = 1).reset_index()

In [94]:
vcounty["county_fips"] = vcounty["county_fips"].astype('int').astype('str')

vcounty["county_fips"] = vcounty["county_fips"].apply(lambda x: "0" + x if len(x) == 4 else x)

In [95]:
vcounty.fillna(0, inplace=True)
vcounty["fips"] = vcounty["county_fips"].astype('int')
vcounty["DEMOCRAT"] = vcounty["DEMOCRAT"]/vcounty["totalvotes"]
vcounty["GREEN"] = vcounty["GREEN"]/vcounty["totalvotes"]
vcounty["LIBERTARIAN"] = vcounty["LIBERTARIAN"]/vcounty["totalvotes"]
vcounty["OTHER"] = vcounty["OTHER"]/vcounty["totalvotes"]
vcounty["REPUBLICAN"] = vcounty["REPUBLICAN"]/vcounty["totalvotes"]
vcounty["vot_percentage"] = vcounty

In [96]:
vcounty = vcounty.applymap(lambda s: s.title() if type(s) == str else s)
vcounty.to_csv("viz_data.csv", index = False)